In [1]:
%cd ..

/home/jovyan/work/spectrum/c_final/v1


In [2]:
import pandas as pd
import base
import os

2024-02-27 08:50:59.184078: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-27 08:50:59.219121: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 08:50:59.219154: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 08:50:59.220012: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-27 08:50:59.225752: I tensorflow/core/platform/cpu_feature_guar

In [3]:
SEED = 42
base.fix_seed(SEED)

desired_directory = '/home/jovyan/work/spectrum/'
os.chdir(desired_directory)

In [3]:
df = base.import_sqlite3_data('Data/01_sqlite_data/ms2_lipid2.db')

In [16]:
df['dataset'].str[-8:].value_counts()

dataset
_Pos.txt    50677
_Neg.txt    31778
Name: count, dtype: int64

In [18]:
len(df['dataset'].str[:-8].unique())

82

In [5]:
df = base.import_sqlite3_data('Data/01_sqlite_data/ms2_lipid2.db')
df = df[df['Comment'].str.lower() == "ok"]
df = df[~df['Metabolitename'].str.contains('\(d')]

In [6]:
df_pos = df[df['Adducttype'].str.contains('\]\+') | df['Adducttype'].str.contains('\]2\+')]
df_neg = df[df['Adducttype'].str.contains('\]\-') | df['Adducttype'].str.contains('\]2\-')]
print(len(df), len(df_pos)+len(df_neg), len(df_pos), len(df_neg))

39864 39864 19726 20138


In [7]:
df_pos.to_csv('Data/02_basedata/pos/df_pos.csv' )
df_neg.to_csv('Data/02_basedata/neg/df_neg.csv')

converted horizontally

In [9]:
df_pos_exp_ = base.spectrum_tidy(df_pos)
df_pos_neuloss_ = base.neutralloss_tidy(df_pos)
df_pos_data = base.spectrum_neutralloss_table(df_pos_exp_, df_pos_neuloss_)#, df_pos_remainder_)

df_pos_data_ = pd.merge(df_pos_data, df[['Metabolitename','Ontology','dataset','AlignmentID','AverageMz']],\
                        on=['Metabolitename','Ontology','dataset','AlignmentID','AverageMz'], how='left')
df_pos_data_ = df_pos_data_[~df_pos_data_['Metabolitename'].str.contains("also known as")]. \
               set_index(['Metabolitename','Ontology','dataset','AlignmentID','AverageMz'])

In [11]:
df_neg_exp_ = base.spectrum_tidy(df_neg)
df_neg_neuloss_ = base.neutralloss_tidy(df_neg)
df_neg_data = base.spectrum_neutralloss_table(df_neg_exp_, df_neg_neuloss_)

df_neg_data_ = pd.merge(df_neg_data, df[['Metabolitename','Ontology','dataset','AlignmentID','AverageMz']],\
                        on=['Metabolitename','Ontology','dataset','AlignmentID','AverageMz'], how='left')
df_neg_data_ = df_neg_data_[~df_neg_data_['Metabolitename'].str.contains("also known as")]. \
               set_index(['Metabolitename','Ontology','dataset','AlignmentID','AverageMz'])

In [11]:
df_pos_data_.to_csv('Data/02_basedata/pos/df_pos_data_.csv')
df_neg_data_.to_csv('Data/02_basedata/neg/df_neg_data_.csv')
df.to_csv('Data/02_basedata/df.csv')